Building an example of a map & interaction I'm looking for, using ipyleaflet.
May not include the breed selector

In [1]:
from ipyleaflet import Map, GeoData, Choropleth, basemaps, LayersControl
import geopandas as gpd
import pandas as pd
import json

from pathlib import Path

from branca.colormap import linear

from ipywidgets import Text, HTML, Dropdown, Output
from ipyleaflet import WidgetControl, GeoJSON


In [2]:
geo_filepath = Path.cwd().joinpath('data').joinpath('clean_seattle_zipcodes.geojson')

In [3]:
gdf = gpd.read_file(geo_filepath)
gdf.zipcode = gdf.zipcode.astype('int64')

with open(geo_filepath, 'r') as f:
   geodata = json.load(f)

In [4]:
df = pd.read_csv(Path.cwd().joinpath('data').joinpath('seattle_dogs_single_breed.csv'))

In [5]:
top_breeds_by_zip_df = pd.read_csv(Path.cwd().joinpath('data').joinpath('top_5_breeds_by_zip.csv'))

In [6]:
gdf = gdf.merge(top_breeds_by_zip_df, how='inner', left_on='zipcode', right_on='zip')

# keep only relevant columns
gdf = gdf[['zip', 'geometry', 'top_5_breeds']]


In [7]:
# breed_dropdown = Dropdown(
#     options=sorted(list(df.breed.unique())),
#     # value=df.breed.unique(),
#     description='breed:',
#     disabled=False,
# )
# display(breed_dropdown)

In [21]:
# labrador retriever, golden retriever, aussies are most common
selected_breed = 'labrador retriever'

raw_df = df[df.breed==selected_breed][['zip', 'count']]
gdf = gdf.merge(raw_df, how='inner', on='zip')
gdf.rename(columns={'count':f"{selected_breed}_count"}, inplace=True)
# gdf = gdf[['zip', 'geometry', 'top_5_breeds']]
# gdf.head()

gdf = gdf[['zip', 'geometry', f"{selected_breed}_count", "top_5_breeds"]]


In [9]:
# and for choropleth, need json things
# Need all the data
df.to_dict()
raw = raw_df.to_dict('records') # list[dict]
choro_data = {}

for elem in raw:
    choro_data[str(elem['zip'])] = elem['count']

# add zeros to zips that are not in the resulting dataset:
for elem in geodata['features']:
    if elem['id'] not in choro_data.keys():
        choro_data[elem['id']] = 0

In [18]:
m = Map(center=(47.6062, -122.33207), zoom=10)

layer = Choropleth(
    geo_data = geodata,
    choro_data = choro_data,
    colormap = linear.PRGn_11,
    nan_opacity=0,
    border_color='black',
    style = {'fillOpacity':0.3 }
)

m.add(layer)

# click layer
geo_data = GeoData(
    geo_dataframe=gdf,
    style={
        "color": "black",
        "fillColor": "#366370",
        "opacity": 0, # 0.05,
        "weight": 1.9,
        "dashArray": "2",
        "fillOpacity": 0, # 0.4,
    },
    hover_style={"fillColor": "#b08a3e", "fillOpacity": 0.1},
)
m.add(geo_data)


html = HTML("""Hover over""")
html.layout.margin = "0px 20px 20px 20px"
control = WidgetControl(widget=html, position="bottomright")
m.add(control)

def update_html(feature, **kwargs):
    html.value = f"""{feature["properties"]}
    """
    
geo_data.on_hover(update_html)


In [20]:
m

Map(bottom=91785.0, center=[47.57559927673143, -122.20840189691685], controls=(ZoomControl(options=['position'…